In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set() # all plot will cover on seaborn style

In [78]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [79]:
df_train.shape

(188533, 13)

In [80]:
# getting descriptive stats
df_train.describe(include='all')
# use include = all for getting categorical column also.

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
count,188533.000000,188533,188533,188533.000000,188533.000000,183450,188533,188533,188533,188533,186081,167114,1.885330e+05
unique,NaN,57,1897,NaN,NaN,7,1117,52,319,156,2,1,NaN
top,NaN,Ford,F-150 XLT,NaN,NaN,Gasoline,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,A/T,Black,Black,None reported,Yes,NaN
freq,NaN,23088,2945,NaN,NaN,165940,3462,49904,48658,107674,144514,167114,NaN
mean,94266.000000,NaN,NaN,2015.829998,65705.295174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.387802e+04
std,54424.933488,NaN,NaN,5.660967,49798.158076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.881952e+04
min,0.000000,NaN,NaN,1974.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+03
25%,47133.000000,NaN,NaN,2013.000000,24115.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000e+04
50%,94266.000000,NaN,NaN,2017.000000,57785.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.082500e+04
75%,141399.000000,NaN,NaN,2020.000000,95400.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.990000e+04


In [81]:
df_train.drop('id', axis=1, inplace=True)             # not important 
df_train.drop('clean_title', axis=1, inplace=True)    # not important as only one unique variable
df_train.drop('ext_col', axis=1, inplace=True)        # not relevent realisticly
df_train.drop('int_col', axis=1, inplace=True)        # not relevent realisticly
df_train.drop('fuel_type', axis=1, inplace=True)      # dropped beacuse will create new fuel_type from engine column

In [82]:
import pandas as pd

# Step 1: Filter out cars older than model year 2005
df_train = df_train[df_train['model_year'] >= 2005].copy()

# Step 2: IQR-based outlier removal function
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply IQR method on relevant numerical columns
for col in ['price', 'milage']:
    df_train = remove_outliers_iqr(df_train, col)

# Show resulting shape and summary of selected numeric columns
df_train.shape, df_train[['model_year', 'milage', 'price']].describe()


((165953, 8),
           model_year         milage          price
 count  165953.000000  165953.000000  165953.000000
 mean     2016.435672   64081.212259   34496.691846
 std         4.615815   44937.804209   20952.376399
 min      2005.000000     100.000000    2000.000000
 25%      2014.000000   25997.000000   17900.000000
 50%      2017.000000   57939.000000   30000.000000
 75%      2020.000000   92000.000000   46500.000000
 max      2024.000000  194500.000000   97500.000000)

In [83]:
df_train.shape

(165953, 8)

In [84]:
# Check missing values in each column
missing_values = df_train.isnull().sum()

# Show all columns with their missing value counts
print(missing_values)

brand              0
model              0
model_year         0
milage             0
engine             0
transmission       0
accident        2115
price              0
dtype: int64


In [85]:
# Count unique values in each column of df_train
unique_counts = df_train.nunique().sort_values(ascending=False)

print(unique_counts)

milage          6065
model           1893
price           1363
engine          1117
brand             56
transmission      52
model_year        20
accident           2
dtype: int64


In [86]:
# Loop through each object column and print unique values
for col in df_train.select_dtypes(include='object').columns:
    print(f"\nColumn: {col}")
    print(df_train[col].unique())



Column: brand
['Genesis' 'Mercedes-Benz' 'Audi' 'Chevrolet' 'Ford' 'BMW' 'Tesla'
 'Cadillac' 'Land' 'GMC' 'Toyota' 'Hyundai' 'Volvo' 'Volkswagen' 'Buick'
 'RAM' 'Hummer' 'Alfa' 'INFINITI' 'Jeep' 'Porsche' 'McLaren' 'Honda'
 'MINI' 'Lexus' 'Dodge' 'Nissan' 'Jaguar' 'Acura' 'Lincoln' 'Kia'
 'Mitsubishi' 'Rolls-Royce' 'Maserati' 'Pontiac' 'Saturn' 'Bentley'
 'Mazda' 'Subaru' 'Chrysler' 'Rivian' 'Lamborghini' 'Ferrari' 'Aston'
 'Lucid' 'Lotus' 'Scion' 'smart' 'Karma' 'Suzuki' 'FIAT' 'Saab' 'Mercury'
 'Polestar' 'Maybach' 'Bugatti']

Column: model
['G90 5.0 Ultimate' 'Metris Base' 'A6 2.0T Sport' ...
 'Caprice Classic Base' 'IONIQ Plug-In Hybrid SEL' 'X5 3.0si']

Column: engine
['420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel'
 '208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel'
 '252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel' ...
 '139.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas'
 '115.0HP 2.0L 4 Cylinder Engine Gasoline Fuel'
 '313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas']

Column: t

In [87]:
df_train['accident'] = df_train['accident'].fillna('None reported')

In [88]:
print(df_train['accident'].unique())

['None reported' 'At least 1 accident or damage reported']


In [89]:
import re

# Function to extract engine details
def extract_engine_info(text):
    hp = re.search(r'([\d.]+)HP', text)
    size = re.search(r'([\d.]+)L', text)
    cyl = re.search(r'(\d+)\sCylinder', text)
    fuel = re.search(r'Engine\s(.+?)$', text)

    return pd.Series({
        'engine_hp': float(hp.group(1)) if hp else None,
        'engine_size_L': float(size.group(1)) if size else None,
        'engine_cylinders': int(cyl.group(1)) if cyl else None,
        'engine_fuel_type': fuel.group(1).strip() if fuel else None
    })

# Apply the function to create new columns
df_train[['engine_hp', 'engine_size_L', 'engine_cylinders', 'engine_fuel_type']] = df_train['engine'].apply(extract_engine_info)

# Display the first few rows to verify
df_train[['engine', 'engine_hp', 'engine_size_L', 'engine_cylinders', 'engine_fuel_type']].head()


,engine,engine_hp,engine_size_L,engine_cylinders,engine_fuel_type
3,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,420.0,5.0,8.0,Gasoline Fuel
4,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,208.0,2.0,4.0,Gasoline Fuel
5,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,252.0,2.0,4.0,Gasoline Fuel
6,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,333.0,3.0,6.0,Gasoline Fuel
7,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,355.0,5.3,8.0,Flex Fuel Capability


In [90]:
# now we can drop engine column
df_train.drop('engine', axis=1, inplace=True) 

In [91]:
# check if there is missing values in any new column
# Check missing values in the newly created engine-related columns
missing_engine_info = df_train[['engine_hp', 'engine_size_L', 'engine_cylinders', 'engine_fuel_type']].isnull().sum()

print(missing_engine_info)


engine_hp           28990
engine_size_L       12534
engine_cylinders    33163
engine_fuel_type    33217
dtype: int64


In [92]:
# Group-wise imputation using brand, model, model_year
group_cols = ['brand', 'model', 'model_year']

# Fill numeric features with group-wise median
for col in ['engine_hp', 'engine_size_L', 'engine_cylinders']:
    df_train[col] = df_train.groupby(group_cols)[col].transform(lambda x: x.fillna(x.median()))

# Fill categorical feature with group-wise mode
def fill_mode(series):
    mode_val = series.mode()
    return series.fillna(mode_val[0]) if not mode_val.empty else series

df_train['engine_fuel_type'] = df_train.groupby(group_cols)['engine_fuel_type'].transform(fill_mode)

# Step 4: Fill any remaining missing values with global median/mode
df_train['engine_hp'].fillna(df_train['engine_hp'].median(), inplace=True)
df_train['engine_size_L'].fillna(df_train['engine_size_L'].median(), inplace=True)
df_train['engine_cylinders'].fillna(df_train['engine_cylinders'].median(), inplace=True)
df_train['engine_fuel_type'].fillna(df_train['engine_fuel_type'].mode()[0], inplace=True)

# Step 5: Show remaining missing values
missing_after_fill = df_train[['engine_hp', 'engine_size_L', 'engine_cylinders', 'engine_fuel_type']].isnull().sum()
missing_after_fill


engine_hp           0
engine_size_L       0
engine_cylinders    0
engine_fuel_type    0
dtype: int64

The code extracted key numerical and categorical features from the engine column, such as horsepower, engine size, cylinder count, and fuel type. Missing values in these new columns were filled using a hybrid approach: first, the dataset was grouped by brand, model, and model_year to impute missing values using the median (for numeric features) or mode (for categorical features) within each group of similar cars. Any remaining missing values were then filled using the overall median or mode of the entire dataset. This approach preserved data quality while minimizing loss from missing entries.

In [93]:
df_train.head()

,brand,model,model_year,milage,transmission,accident,price,engine_hp,engine_size_L,engine_cylinders,engine_fuel_type
3,Genesis,G90 5.0 Ultimate,2017,19500,Transmission w/Dual Shift Mode,None reported,45000,420.0,5.0,8.0,Gasoline Fuel
4,Mercedes-Benz,Metris Base,2021,7388,7-Speed A/T,None reported,97500,208.0,2.0,4.0,Gasoline Fuel
5,Audi,A6 2.0T Sport,2018,40950,A/T,None reported,29950,252.0,2.0,4.0,Gasoline Fuel
6,Audi,A8 L 3.0T,2016,62200,8-Speed A/T,None reported,28500,333.0,3.0,6.0,Gasoline Fuel
7,Chevrolet,Silverado 1500 1LZ,2016,102604,A/T,None reported,12500,355.0,5.3,8.0,Flex Fuel Capability


In [94]:
# Columns to one-hot encode
categorical_cols = ['brand', 'transmission', 'accident', 'engine_fuel_type']

# One-hot encoding using pd.get_dummies
df_train = pd.get_dummies(df_train, columns=categorical_cols, drop_first=True)

# Display the first few rows to verify
df_train.head()


,model,model_year,milage,price,engine_hp,engine_size_L,engine_cylinders,brand_Alfa,brand_Aston,brand_Audi,...,transmission_Transmission Overdrive Switch,transmission_Transmission w/Dual Shift Mode,transmission_Variable,transmission_–,accident_None reported,engine_fuel_type_Flex Fuel Capability,engine_fuel_type_Gas/Electric Hybrid,engine_fuel_type_Gasoline Fuel,engine_fuel_type_Gasoline/Mild Electric Hybrid,engine_fuel_type_Plug-In Electric/Gas
3,G90 5.0 Ultimate,2017,19500,45000,420.0,5.0,8.0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,Metris Base,2021,7388,97500,208.0,2.0,4.0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
5,A6 2.0T Sport,2018,40950,29950,252.0,2.0,4.0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
6,A8 L 3.0T,2016,62200,28500,333.0,3.0,6.0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
7,Silverado 1500 1LZ,2016,102604,12500,355.0,5.3,8.0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [95]:
# Check data types of all columns
print("Column Data Types:\n")
print(df_train.dtypes)

# Check for missing values in each column
print("\nMissing Values in Each Column:\n")
print(df_train.isnull().sum())


Column Data Types:

model                                              object
model_year                                          int64
milage                                              int64
price                                               int64
engine_hp                                         float64
                                                   ...   
engine_fuel_type_Flex Fuel Capability               uint8
engine_fuel_type_Gas/Electric Hybrid                uint8
engine_fuel_type_Gasoline Fuel                      uint8
engine_fuel_type_Gasoline/Mild Electric Hybrid      uint8
engine_fuel_type_Plug-In Electric/Gas               uint8
Length: 119, dtype: object

Missing Values in Each Column:

model                                             0
model_year                                        0
milage                                            0
price                                             0
engine_hp                                         0
                      

In [96]:
from sklearn.preprocessing import StandardScaler

# List of numerical features to scale (exclude one-hot and target)
numeric_features = ['milage', 'engine_hp', 'engine_size_L', 'engine_cylinders', 'model_year']

# Initialize scaler
scaler = StandardScaler()

# Fit and transform
df_train[numeric_features] = scaler.fit_transform(df_train[numeric_features])

# Preview the scaled data
df_train[numeric_features].head()


,milage,engine_hp,engine_size_L,engine_cylinders,model_year
3,-0.992068,0.704307,0.938423,1.147308,0.122260
4,-1.261597,-1.254525,-1.290292,-1.517057,0.988849
5,-0.514740,-0.847975,-1.290292,-1.517057,0.338907
6,-0.041863,-0.099554,-0.547387,-0.184875,-0.094387
7,0.857249,0.103721,1.161294,1.147308,-0.094387


In [97]:
# Target encoding: replace 'model' with average price per model
model_target_mean = df_train.groupby('model')['price'].mean()

# Map to a new column
df_train['model_encoded'] = df_train['model'].map(model_target_mean)

# Drop original model column (optional)
df_train.drop(columns=['model'], inplace=True)

# Check the result
df_train[['model_encoded']].head()


,model_encoded
3,41225.919540
4,44870.627409
5,28007.062500
6,23966.184211
7,24076.636364


In [98]:
df_train

,model_year,milage,price,engine_hp,engine_size_L,engine_cylinders,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,...,transmission_Transmission w/Dual Shift Mode,transmission_Variable,transmission_–,accident_None reported,engine_fuel_type_Flex Fuel Capability,engine_fuel_type_Gas/Electric Hybrid,engine_fuel_type_Gasoline Fuel,engine_fuel_type_Gasoline/Mild Electric Hybrid,engine_fuel_type_Plug-In Electric/Gas,model_encoded
3,0.122260,-0.992068,45000,0.704307,0.938423,1.147308,0,0,0,0,...,1,0,0,1,0,0,1,0,0,41225.919540
4,0.988849,-1.261597,97500,-1.254525,-1.290292,-1.517057,0,0,0,0,...,0,0,0,1,0,0,1,0,0,44870.627409
5,0.338907,-0.514740,29950,-0.847975,-1.290292,-1.517057,0,0,1,0,...,0,0,0,1,0,0,1,0,0,28007.062500
6,-0.094387,-0.041863,28500,-0.099554,-0.547387,-0.184875,0,0,1,0,...,0,0,0,1,0,0,1,0,0,23966.184211
7,-0.094387,0.857249,12500,0.103721,1.161294,1.147308,0,0,0,0,...,0,0,0,1,1,0,0,0,0,24076.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,0.122260,-0.335603,27500,0.704307,1.829909,1.147308,0,0,0,0,...,1,0,0,1,0,0,1,0,0,40703.518797
188529,0.338907,-0.789565,30000,0.380915,-0.547387,-0.184875,0,0,0,0,...,0,0,0,0,0,0,1,0,0,49189.277228
188530,0.988849,-1.122248,86900,1.157056,0.195518,1.147308,0,0,0,0,...,0,0,0,1,0,0,1,0,0,56302.692308
188531,1.205496,-1.116796,84900,0.048283,-0.547387,-0.184875,0,0,1,0,...,0,0,0,1,0,0,1,0,0,34790.822394


## Splitting the dataset and running Linear regression model

In [99]:
from sklearn.model_selection import train_test_split

# First split: train vs (validation + test)
df_temp, df_test = train_test_split(df_train, test_size=0.2, random_state=42)

# Second split: validation from remaining training data
df_train_final, df_val = train_test_split(df_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Now you have:
# 60% train, 20% validation, 20% test


In [100]:
# Define target column
target = 'price'

# Separate features and target for each set
X_train = df_train_final.drop(columns=[target])
y_train = df_train_final[target]

X_val = df_val.drop(columns=[target])
y_val = df_val[target]

X_test = df_test.drop(columns=[target])
y_test = df_test[target]


In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 1. Initialize the model
model = LinearRegression()

# 2. Train the model
model.fit(X_train, y_train)

# 3. Predict on validation set
y_val_pred = model.predict(X_val)

# 4. Evaluate the model
rmse = mean_squared_error(y_val, y_val_pred, squared=False)
r2 = r2_score(y_val, y_val_pred)

mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {mae:.2f}")
print(f"Validation RMSE: {rmse:.2f}")
print(f"Validation R² Score: {r2:.4f}")


Validation MAE: 10203.50
Validation RMSE: 13730.98
Validation R² Score: 0.5715


In [102]:
# Feature importance (coefficients)
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

print(coef_df.head(10))  # Top 10 most impactful features


                                            Feature   Coefficient
10                                    brand_Bugatti  27268.480718
63   transmission_10-Speed Automatic with Overdrive  13713.982654
44                                   brand_Polestar -11234.111064
89                          transmission_8-SPEED AT -10998.814663
85               transmission_7-Speed DCT Automatic -10836.403676
88                         transmission_8-SPEED A/T  10083.862044
101                              transmission_CVT-F -10035.852099
54                                     brand_Suzuki   9722.175443
110                                  transmission_–   9323.799352
64                                   transmission_2  -7603.739036


Validation RMSE: 13730.98

Validation R² Score: 0.5715

Validation MAE: 10203.50